### Portions of this notebook adapted from https://github.com/rouseguy/DeepLearning-NLP/blob/master/notebooks/4.%20Recurrent%20Neural%20Networks.ipynb

In [1]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format(r'F:\NeuralNetwork\word2vec\GoogleNews-vectors-negative300.bin', binary=True)


C:\Users\Stephen\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import keras
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras import backend as K
from keras.engine import Layer
from keras.models import Sequential, Model
from keras.callbacks import TensorBoard, EarlyStopping

from keras.layers import *
from keras import *

Using TensorFlow backend.


In [3]:
from sklearn.cross_validation import train_test_split

C:\Users\Stephen\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
import numpy as np

In [5]:
from nltk.corpus import words as english_words

In [6]:
from string import *
from time import *
from multiprocessing.dummy import Pool



In [7]:
#ew_set = set(english_words.words())
# reg_vocab = [word for word in model.vocab if word in ew_set]
#reg_vocab = [word for word in model.vocab if all(letter in printable for letter in word)]

In [8]:
#reg_vocab = [word for word in model.vocab if word in ew_set]
reg_vocab = ['<' + word + '>' for word in model.vocab if all(letter not in {'#', '@', '.', ',', '/', '\\', '<', '>', *ascii_uppercase} and letter in printable for letter in word)]
reg_vocab_set = set(reg_vocab)

In [9]:
char_list = printable


In [10]:
char_indices = dict((c, i) for i, c in enumerate(char_list))
indices_char = dict((i, c) for i, c in enumerate(char_list))

In [11]:
MAX_LENGTH = 0
i = 0
MAX_INDEX = 0
for i,n in enumerate(reg_vocab):
    
    if len(n) > MAX_LENGTH:
        MAX_LENGTH = len(n)
        MAX_INDEX = i
        
print(MAX_LENGTH,MAX_INDEX)

100 383866


In [12]:
def word_to_char_seq(word):
    word_chars = list(word)
    word_chars_indices = np.array(list(map(lambda char: char_indices[char], word_chars)))
    #return word_chars_indices
    return sequence.pad_sequences([word_chars_indices], maxlen=MAX_LENGTH, padding='post')[0]

In [13]:
def vectorize_words(words, verbose=False):
    vwords =[]
    for name in words:
        x = [char_indices[c] for c in name]
        if verbose:
            print(x)
        vx = np.eye(len(char_indices))[x]  #https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.eye.html
        if verbose:
            print(vx[0:12])
        vwords.append(vx)
    return vwords

In [14]:
y = np.array([model.wv[word[1:-1]] for word in reg_vocab])
#del model

C:\Users\Stephen\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [15]:
#X = np.array([word_to_char_seq(word) for word in reg_vocab], dtype='uint8')

In [16]:
X_train, X_test, y_train, y_test = train_test_split(reg_vocab, y)
#del X
del y
X_train

['<particulate_opacity_mercury>',
 '<laundry_soap>',
 '<perming>',
 '<packaging_heat_shrinkable>',
 '<concerend>',
 '<pepper_spray_rubber_bullets>',
 '<----------------------------------------------------------_liabilities>',
 '<nonintervention>',
 '<rotations_lubrication>',
 '<velveteen_jacket>',
 '<grape_hyacinth>',
 '<defective_painkillers>',
 '<reattach_tendon>',
 '<juicier>',
 '<phospho>',
 '<worry_wart>',
 '<downloaden>',
 '<roosting>',
 '<ultrabright>',
 '<ingest>',
 '<retroviral_vector>',
 '<uncorrupt>',
 '<morphine_tablets>',
 '<unwound>',
 '<microwaving>',
 '<volgens>',
 '<dwarf_conifers>',
 '<fresh_garbanzos>',
 '<dilly_dallies>',
 '<antiwar_sentiments>',
 '<multitrack_recording>',
 '<craniofacial_anomalies>',
 '<starlets>',
 '<halibut_fishing>',
 '<refinery_coker>',
 '<vividly_recollect>',
 '<bustier_dress>',
 '<promo_pic>',
 '<lightning_bolt>',
 '<riskless_profit>',
 '<mistaking>',
 '<creamy_puree>',
 '<sensorium>',
 '<ragworm>',
 '<gender_disparity>',
 '<seemingly_hopeles

In [17]:
X_train = np.array([word_to_char_seq(word) for word in X_train], dtype='uint8')

In [18]:
X_train.shape

(323078, 100)

In [19]:
def cos_plus_absolute(ytrue, ypred):
    return losses.mean_absolute_error(ytrue, ypred) + losses.cosine_proximity(ytrue, ypred)

In [20]:
K.clear_session()

num_classes = len(char_list)
k_model = Sequential()
#k_model.add(OneHot(len(printable)))
#k_model.add(Input(shape=(98,), dtype='uint8'))
#k_model.add(Lambda(K.one_hot, arguments={'num_classes': 100}, input_shape=(98,), output_shape=(98,100)))
# k_model.add(Lambda(to_categorical(100)))(input_shape=(1, 98))
onehot_weights = np.eye(num_classes)
k_model.add(Embedding(num_classes, num_classes,
                      trainable=False,
                      embeddings_initializer=keras.initializers.Identity(gain=1.0),
                      input_length=MAX_LENGTH,
                      #weights=[onehot_weights],
                      name='onehot'))
k_model.add(Conv1D(filters=16, kernel_size=2, strides=1, padding='valid'))
k_model.add(MaxPooling1D(pool_size=2))
k_model.add(Flatten())

k_model.add(Dense(300, activation='relu'))
k_model.compile(loss=cos_plus_absolute,
                optimizer='adam',
                metrics=['accuracy', 'mean_squared_error', 'cosine_proximity', 'mean_absolute_error'])

k_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
onehot (Embedding)           (None, 100, 100)          10000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 99, 16)            3216      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 49, 16)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               235500    
Total params: 248,716
Trainable params: 238,716
Non-trainable params: 10,000
_________________________________________________________________


In [21]:
K.clear_session()
num_classes = len(printable)
#k_model.add(OneHot(len(printable)))
#k_model.add(Input(shape=(98,), dtype='uint8'))
#k_model.add(Lambda(K.one_hot, arguments={'num_classes': 100}, input_shape=(98,), output_shape=(98,100)))
# k_model.add(Lambda(to_categorical(100)))(input_shape=(1, 98))
onehot_weights = np.eye(num_classes)

# k_model.add(Conv1D(filters=16, kernel_size=10 strides=1, padding='valid'))
# k_model.add(MaxPooling1D(pool_size=2))
i = Input(shape=(MAX_LENGTH,))
mask = Masking(0)(i)
one_hot = Embedding(num_classes, num_classes,
                      trainable=False,
                      embeddings_initializer=keras.initializers.Identity(gain=1.0),
                      #weights=[onehot_weights],
                      name='onehot')(mask)
small = Conv1D(filters=8, kernel_size=2, strides=1, padding='same')(one_hot)
med = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(one_hot)
large = Conv1D(filters=16, kernel_size=4, strides=1, padding='same')(one_hot)
vlarge = Conv1D(filters=16, kernel_size=5, strides=1, padding='same')(one_hot)
huge = Conv1D(filters=16, kernel_size=6, strides=1, padding='same')(one_hot)
#hugest = Conv1D(filters=16, kernel_size=7, strides=1, padding='same')(one_hot)

# small_pool = MaxPooling1D(pool_size=2)(small)
# med_pool = MaxPooling1D(pool_size=2)(med)
# large_pool = MaxPooling1D(pool_size=2)(large)

# small_flatten = Flatten()(small_pool)
# med_flatten = Flatten()(med_pool)
# large_flatten = Flatten()(large_pool)

merge = concatenate([small, med, large, vlarge, huge], axis=-1)

pool = MaxPooling1D(pool_size=2)(merge)

mergeConv = Conv1D(filters=64, kernel_size=3, strides=1)(pool)
mergePool = MaxPooling1D(pool_size=2)(mergeConv)
full_flatten = Flatten()(mergePool)
dense1 = Dense(1000, activation='relu')(full_flatten)
dense2 = Dense(500, activation='relu')(dense1)
dense3 = Dense(400, activation='relu')(dense2)


out = Dense(300, activation='tanh', name='out')(dense3)

k_model = Model(inputs=i, outputs=out)

k_model.compile(loss=cos_plus_absolute,
                optimizer='adam',
                metrics=['accuracy', 'mean_squared_error', 'cosine_proximity', 'mean_absolute_error'])

k_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 100)          0           input_1[0][0]                    
__________________________________________________________________________________________________
onehot (Embedding)              (None, 100, 100)     10000       masking_1[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 100, 8)       1608        onehot[0][0]                     
__________________________________________________________________________________________________
conv1d_2 (

In [22]:
tensorboard = TensorBoard(log_dir="logs/{}".format(strftime("%a-%H-%M-%S")))

k_model.fit(X_train, y_train,
                    batch_size=1000,
                    epochs=50,
                    verbose=1,
                    validation_split=0.1,
#                    callbacks=[tensorboard])
                    callbacks=[tensorboard,
                               EarlyStopping(monitor='val_loss',
                                              min_delta=.0001,
                                              patience=3,
                                              verbose=0, mode='auto')])

Train on 290770 samples, validate on 32308 samples
Epoch 1/50
290770/290770 [==============================] - 180s 621us/step - loss: -0.2679 - acc: 0.0345 - mean_squared_error: 0.0208 - cosine_proximity: -0.3707 - mean_absolute_error: 0.1029 - val_loss: -0.2905 - val_acc: 0.0379 - val_mean_squared_error: 0.0200 - val_cosine_proximity: -0.3912 - val_mean_absolute_error: 0.1006
Epoch 2/50
290770/290770 [==============================] - 176s 606us/step - loss: -0.2998 - acc: 0.0432 - mean_squared_error: 0.0202 - cosine_proximity: -0.4008 - mean_absolute_error: 0.1010 - val_loss: -0.3063 - val_acc: 0.0461 - val_mean_squared_error: 0.0198 - val_cosine_proximity: -0.4063 - val_mean_absolute_error: 0.1000
Epoch 3/50
290770/290770 [==============================] - 176s 606us/step - loss: -0.3147 - acc: 0.0481 - mean_squared_error: 0.0200 - cosine_proximity: -0.4151 - mean_absolute_error: 0.1004 - val_loss: -0.3155 - val_acc: 0.0496 - val_mean_squared_error: 0.0197 - val_cosine_proximity: -

In [24]:
from random import sample

# k_model._make_predict_function()
# graph = K.get_session().graph

# def func(word): 
#     print(word)
#     return [item[0] for item in model.most_similar(positive=k_model.predict(np.array([word_to_char_seq(word)])), topn=10000000) if item[0] in reg_vocab_set].index(word)
def makepred(word):
    return k_model.predict(np.array([word_to_char_seq(word[1:-1])]))
def getindex(pred):
    vec = pred[0]
    word = pred[1][1:-1]
    return [item[0] for item in model.most_similar(vec, topn=3000000) if '<' + item[0] + '>' in reg_vocab_set].index(word)

preds = [(makepred(word), word) for word in sample(X_test, 15000)]
#indices = [getindex(vec) for vec in preds]

p = Pool(16)

%time out = list(map(getindex, preds))


Wall time: 21h 32min 20s


In [25]:
from statistics import *
outperc = [i/len(reg_vocab_set) for i in out]
print(mean(outperc))
print(stdev(outperc))

print(median(outperc))

0.3213411453726767
0.280798505402873
0.24494104756355467


In [ ]:
len(X_test)

In [26]:
len(reg_vocab_set)

430771

### ot